In [1]:
import numpy as np
import cv2
import os
import open3d as o3d
import matplotlib.pylab as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img, img_gray)

In [3]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [4]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.8*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return
    

In [5]:
query_img= cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')
template_img = cv2.imread('./images/final_exam/Templates/Template-1.png')
template_img, template_gray = preprocessing(template_img)

while query_img.isOpened() :
    ret, frame = query_img.read()
    query_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if ret :
        detected, drawmatch = feature_object_detection(template_img, template_gray, frame, query_gray, 10)
        
    if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : # this line control the period between image frame
        break

    cv2.imshow('Video frame', frame)
    print(detected)
query_img.release()
cv2.destroyAllWindows()

[[[166 184 177]
  [166 184 177]
  [165 183 176]
  ...
  [122 137 133]
  [123 138 134]
  [123 138 134]]

 [[166 184 177]
  [165 183 176]
  [165 183 176]
  ...
  [122 137 133]
  [123 138 134]
  [123 138 134]]

 [[165 183 176]
  [165 183 176]
  [164 182 175]
  ...
  [122 137 133]
  [123 138 134]
  [123 138 134]]

 ...

 [[ 84  90  85]
  [ 84  90  85]
  [ 83  89  84]
  ...
  [ 14  12   8]
  [ 14  12   8]
  [ 14  12   8]]

 [[ 78  88  80]
  [ 78  88  80]
  [ 78  88  80]
  ...
  [ 14  12   8]
  [ 14  12   8]
  [ 14  12   8]]

 [[ 72  82  74]
  [ 73  83  75]
  [ 74  84  76]
  ...
  [ 14  12   8]
  [ 14  12   8]
  [ 14  12   8]]]
[[[166 184 177]
  [166 184 177]
  [165 183 176]
  ...
  [124 137 131]
  [125 138 132]
  [125 138 132]]

 [[166 184 177]
  [165 183 176]
  [165 183 176]
  ...
  [124 137 131]
  [125 138 132]
  [125 138 132]]

 [[165 183 176]
  [165 183 176]
  [164 182 175]
  ...
  [124 137 131]
  [125 138 132]
  [125 138 132]]

 ...

 [[ 82  87  85]
  [ 83  88  86]
  [ 82  87  85]
  ..